# Notebook Overview
This Notebook takes the frames produced by `02_Evaluate-Data` as well as data provided by team members and creates final videos useful for result comparison and analysis. Merged plots and video outputs appear in the `../data/06_merged-plots` directory.

This is by far the "least polished" Notebook of the bunch (i.e. lots of awkward variable names and magic numbers). This is because this code used to make one-off videos and graphics for our presentation/report. The code is included for reference. Any of the projects comparison videos/graphics can be produced again from this notebook with minimal edits.

In [ ]:
import cv2
import numpy as np
import sys
import os
from tqdm.notebook import tqdm

# Constants

In [ ]:
INDEX_FRAMES = 254 # from 0 - 253 inclusive
MERGE_TEMPLATE = '../data/06_merged-plots/%06d.jpg'
ANDREW_TEMPLATE = '../data/04_plots/%06d.jpg'
ABBIE_TEMPLATE = '../data/05_team-plots/abbie/%06d.jpg'
SHILOH_TEMPLATE = '../data/05_team-plots/shiloh/%06d.jpg'
SANDRO_TEMPLATE = '../data/05_team-plots/sandro/%06d.jpg'
SANDRO_INFILE = '../data/misc/run5.mp4'
SANDRO_FRAMES = 2099

# Extract Individual Frames from Sandro's Recording

In [ ]:
cap = cv2.VideoCapture(SANDRO_INFILE)

frame_count = 0
while True:
    succ, frame = cap.read()
    if not succ:
        break
    
    window = frame[28:881,254:1385]
    resized = cv2.resize(window, (640,480))
        
    OUTFILE = SANDRO_TEMPLATE % frame_count
    if frame_count % 100 == 0:
        print('Writing frame:', OUTFILE)
    cv2.imwrite(OUTFILE,resized)
    frame_count += 1

cap.release()

# Frame Sync Helper
To get equally spaced sampled raw data from CARLA, the animation loop could not be regulated at a continuous rate. Thus the recording of the data collection Sandro produced did not maintain a consistent frame rate related to the data logs. The values in `sandro_stitch` and `others_stitch` were hand selected as "anchor points" matching frames in Sandro's data collect recording to the analysis plots produced by these scripts and others. Values between anchor points are linearly interpolated to keep the overall video approximately synchronized throughout.

In [ ]:
# sandro_stitch: frame 
sandro_stitch = [8,233,318,497,676,886,1057,1104,1765,2065, 2100]
others_stitch = [0, 30, 40, 60, 82,106, 129, 136, 221, 261,  270]

# correct frames to my internal frame count
others_stitch = [x-9 for x in others_stitch]

def get_sandro_idx(t):
    i = 0
    while t >= others_stitch[i]:
        i += 1
    
    l = i - 1
    h = i
    
    ratio = (sandro_stitch[h] - sandro_stitch[l]) / (others_stitch[h] - others_stitch[l])
    val = ratio * (t - others_stitch[l]) + sandro_stitch[l]
    return int(val)

# Merge All
Merges Sandro's data collect recording and the three analysis frames into a single frame. Writes frames to `../data/06_merged-plots`.

In [ ]:
# merge all

for t in tqdm(range(INDEX_FRAMES)):
    #sandro_idx = int(SANDRO_FRAMES * t / (INDEX_FRAMES - 1))
    #sandro_idx = int(70/9 * (t+8))
    sandro_idx = get_sandro_idx(t)
    top_left = cv2.imread(SANDRO_TEMPLATE % sandro_idx)
    top_right = cv2.imread(ANDREW_TEMPLATE % t)
    bot_left = cv2.imread(ABBIE_TEMPLATE % t)
    bot_right = cv2.imread(SHILOH_TEMPLATE % t)
        
    assert top_left.shape == top_right.shape == bot_left.shape == bot_right.shape
    
    frame = np.zeros((20+480*2,640*2,3))
    frame[:,:,:] = 255
    frame[0:480,0:640,:] = top_left
    frame[0:480,640:,:] = top_right
    frame[20+480:,0:640,:] = bot_left
    frame[20+480:,640:,:] = bot_right
    
    frame[13:50,18:288] = 255
    
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    fontScale              = 1
    fontColor              = (0,0,0)
    lineType               = 2
        
    cv2.putText(frame, 'CARLA Recording', (20, 40), font, fontScale, fontColor, lineType)
    cv2.putText(frame, 'Trajectron++', (640+20, 40), font, fontScale, fontColor, lineType)
    cv2.putText(frame, 'Constant Velocity Model (CVM)', (20, 480+40), font, fontScale, fontColor, lineType)
    cv2.putText(frame, 'CVM + Angular Velocity', (640+20, 480+40), font, fontScale, fontColor, lineType)
    
    cv2.imwrite(MERGE_TEMPLATE % t, frame)    

# Merge One
Merges Sandro's data collect recording with one of the three analysis frames into a single frame. Writes frames to `../data/06_merged-plots`. Warning: If run after previous cell, it will overwrite your results.

In [ ]:
# merge one

for t in tqdm(range(INDEX_FRAMES)):
    sandro_idx = get_sandro_idx(t)
    left = cv2.imread(SANDRO_TEMPLATE % sandro_idx)
    right = cv2.imread(ANDREW_TEMPLATE % t)
    #right = cv2.imread(ABBIE_TEMPLATE % t)
    #right = cv2.imread(SHILOH_TEMPLATE % t)
    
    assert top_left.shape == top_right.shape == bot_left.shape == bot_right.shape
    
    frame = np.zeros((480,640*2,3))
    frame[:,0:640,:] = left
    frame[:,640:,:] = right
    
    cv2.imwrite(MERGE_TEMPLATE % t, frame)    

# FFMPEG
These commands run `ffmpeg` in the `../data/06_merged-plots` directory to create video files from all the individual plots. Google Slides cannot play videos at 8fps, so the second command converts the video to a more standard 30fps.

In [ ]:
!ffmpeg -y -framerate 8 -i ../data/06_merged-plots/%06d.jpg ../data/06_merged-plots/video_8fps.mp4

In [ ]:
!ffmpeg -y -i ../data/06_merged-plots/video_8fps.mp4 -r 30 ../data/06_merged-plots/video.mp4